In [15]:
# basic imports
import os
import matplotlib.pyplot as plt
# display matplotlib graphics in notebook
%matplotlib inline 
import seaborn as sns

# disable warnings for libraries
import warnings
warnings.filterwarnings("ignore")

# configure logger
import logging
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')
logger = logging.getLogger(__name__)

In [21]:
import numpy as np
#Expected rate for each word in the vocabulary
def load_embeddings():
    # Load rates
    logger.info("Loading embedding vectors...")
    rates= np.genfromtxt('/Users/suzie/Downloads/aclImdb/imdbEr.txt')
    logger.info(f"Loaded vectors with shape: {rates.shape}")
    
    # Load associated words
    logger.info("Loading words...")
    with open('/Users/suzie/Downloads/aclImdb/imdb.vocab', 'r', encoding='utf-8') as f:
        words = [line.strip() for line in f.readlines()]
    logger.info(f"Loaded {len(words)} words")
    
    # Verify sizes match
    if len(words) != rates.shape[0]:
        raise ValueError(f"Mismatch between number of words ({len(words)}) and vectors ({rates.shape[0]})")
    
    logger.info("Verification complete - sizes match!")
    
    return rates, words

# Load the data
rates, words = load_embeddings()

# Display some basic information
print("\nDataset information:")
print(f"Number of words: {len(words)}")
print(f"Embedding dimension: {rates.shape}")
print(f"\nFirst few words and their expected rates {[(word, rate) for (word, rate) in zip(words[:5], rates[:5])]}")


06:54:26 INFO:Loading embedding vectors...
06:54:26 INFO:Loaded vectors with shape: (89527,)
06:54:26 INFO:Loading words...
06:54:26 INFO:Loaded 89527 words
06:54:26 INFO:Verification complete - sizes match!



Dataset information:
Number of words: 89527
Embedding dimension: (89527,)

First few words and their expected rates [('the', np.float64(0.0490972013402)), ('and', np.float64(0.201363575849)), ('a', np.float64(0.0333946807184)), ('of', np.float64(0.099837669572)), ('to', np.float64(-0.0790210365788))]


In [26]:
import os
import pandas as pd

def process_movie_reviews(directory_path):
    """
    Traite les fichiers de critiques de films et les combine dans un DataFrame.
    
    Args:
        directory_path (str): Chemin vers le répertoire contenant les fichiers texte
        
    Returns:
        pd.DataFrame: DataFrame contenant les colonnes id, rate, et comment
    """
    # Listes pour stocker les données
    movie_ids = []
    rates = []
    comments = []
    
    # Parcourir tous les fichiers du répertoire
    for filename in os.listdir(directory_path):
        if filename.endswith('.txt'):
            # Extraire l'id et la note du nom du fichier
            parts = filename.split('_')
            if len(parts) == 2:
                movie_id = int(parts[0])
                rate = int(parts[1].split('.')[0])
                
                # Lire le contenu du fichier (commentaire)
                with open(os.path.join(directory_path, filename), 'r', encoding='utf-8') as f:
                    comment = f.read().strip()
                
                # Ajouter les données aux listes
                movie_ids.append(movie_id)
                rates.append(rate)
                comments.append(comment)
    
    # Créer le DataFrame
    df = pd.DataFrame({
        'id': movie_ids,
        'rate': rates,
        'comment': comments
    })
    
    # Trier par id de film
    df = df.sort_values('id')
    
    return df

# Exemple d'utilisation
pos_df = process_movie_reviews('/Users/suzie/Downloads/aclImdb/train/pos')
neg_df = process_movie_reviews('/Users/suzie/Downloads/aclImdb/train/neg')
# Ajouter une colonne 'sentiment' pour distinguer les avis positifs et négatifs 
pos_df['sentiment'] = 1
neg_df['sentiment'] = 0
# Concaténer les DataFrames positifs et négatifs
df = pd.concat([pos_df, neg_df], ignore_index=True)